In [112]:
import pandas as pd
import numpy as np
import statsmodels.api as sm


In [113]:
data=pd.read_csv("../merged/len3_ndb_agg_blp_DropNever.csv",index_col=0,encoding="shift-jis")
data.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,薬価,後発品区分,総計,year,in_hospital,...,id_399.0,id_441.0,id_449.0,id_625.0,id_629.0,id_799.0,q_share,r_share,generic_share_q,generic_share_r
0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,292190.95,2014,1,...,0,0,0,0,0,0,0.000085,0.000036,0.236031,0.053874
1,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,128832.00,2014,1,...,0,0,0,0,0,0,0.000038,0.000016,0.236031,0.053874
2,114.0,解熱鎮痛消炎剤,661140020.0,インドメタシン坐剤 ５０ｍｇ,1145700J3012,19.3,0.0,106328.00,2014,1,...,0,0,0,0,0,0,0.000031,0.000013,0.236031,0.053874
3,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,658257.00,2014,0,...,0,0,0,0,0,0,0.000192,0.000081,0.236031,0.053874
4,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,418727.50,2014,0,...,0,0,0,0,0,0,0.000122,0.000051,0.236031,0.053874


In [114]:
data=data.drop_duplicates(subset=["薬効分類","year"])
# drop never treated group
data.head(),data.shape

(      薬効分類   薬効分類名称       医薬品コード            医薬品名  薬価基準収載医薬品コード    薬価  後発品区分  \
 0    114.0  解熱鎮痛消炎剤  661140242.0  インテバン坐剤５０　５０ｍｇ  1145700J3012  19.3    0.0   
 85   121.0    局所麻酔剤  620005991.0      ストロカイン錠５ｍｇ  1219002F1065   5.7    0.0   
 101  123.0    自律神経剤  620002526.0    チアトンカプセル１０ｍｇ  1231013M2179  15.2    0.0   
 153  124.0     鎮けい剤  620008704.0      ブスコパン錠１０ｍｇ  1242002F1330   7.2    0.0   
 213  131.0     眼科用剤  620207901.0     アダプチノール錠５ｍｇ  1319004F1035  45.9    0.0   
 
                総計  year  in_hospital  ...  id_399.0  id_441.0  id_449.0  \
 0    2.921910e+05  2014            1  ...         0         0         0   
 85   1.161969e+07  2014            1  ...         0         0         0   
 101  9.128303e+06  2014            1  ...         0         0         0   
 153  1.495471e+07  2014            1  ...         0         0         0   
 213  5.229358e+06  2014            1  ...         0         0         0   
 
     id_625.0 id_629.0 id_799.0   q_share   r_share  generic

In [115]:
set(data["薬効分類"])

{114.0,
 121.0,
 123.0,
 124.0,
 131.0,
 132.0,
 211.0,
 218.0,
 222.0,
 223.0,
 224.0,
 225.0,
 229.0,
 231.0,
 232.0,
 235.0,
 239.0,
 245.0,
 252.0,
 259.0,
 264.0,
 265.0,
 313.0,
 321.0,
 332.0,
 339.0,
 391.0,
 399.0,
 441.0,
 449.0,
 625.0,
 629.0,
 799.0}

In [116]:
data=data.loc[:,["薬効分類","year",'generic_share_q', 'generic_share_r']]
data.columns

Index(['薬効分類', 'year', 'generic_share_q', 'generic_share_r'], dtype='object')

In [117]:
# 33 ids exist
add_df=pd.DataFrame(columns=data.columns)
for id in set(data["薬効分類"]):
    for year in range(2008,2024):
        if year not in data[data["薬効分類"]==id]["year"].values:
            add_df=add_df.append(pd.DataFrame([[id,year]+[np.nan]*(data.shape[1]-2)],columns=data.columns),ignore_index=True)
add_df.sort_values(by=["薬効分類","year"],inplace=True)
add_df.reset_index(drop=True,inplace=True)
add_df,add_df.shape

C:\Users\hakos\AppData\Local\Temp\ipykernel_11172\2703796558.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  add_df=add_df.append(pd.DataFrame([[id,year]+[np.nan]*(data.shape[1]-2)],columns=data.columns),ignore_index=True)


(      薬効分類  year generic_share_q generic_share_r
 0    114.0  2008             NaN             NaN
 1    114.0  2009             NaN             NaN
 2    114.0  2010             NaN             NaN
 3    114.0  2011             NaN             NaN
 4    114.0  2012             NaN             NaN
 ..     ...   ...             ...             ...
 259  799.0  2011             NaN             NaN
 260  799.0  2012             NaN             NaN
 261  799.0  2013             NaN             NaN
 262  799.0  2022             NaN             NaN
 263  799.0  2023             NaN             NaN
 
 [264 rows x 4 columns],
 (264, 4))

In [118]:
dummies=pd.get_dummies(data["薬効分類"])
data=pd.concat([data,dummies],axis=1)
data

,薬効分類,year,generic_share_q,generic_share_r,114.0,121.0,123.0,124.0,131.0,132.0,...,321.0,332.0,339.0,391.0,399.0,441.0,449.0,625.0,629.0,799.0
0,114.0,2014,0.236031,0.053874,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
85,121.0,2014,0.062114,0.053451,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
101,123.0,2014,0.129137,0.064243,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
153,124.0,2014,0.436744,0.216412,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
213,131.0,2014,0.136869,0.080471,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29129,441.0,2021,0.342768,0.263456,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
29169,449.0,2021,0.608051,0.363450,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
29373,625.0,2021,0.502607,0.052158,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
29616,629.0,2021,0.284349,0.073842,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [119]:
dummies=pd.get_dummies(add_df["薬効分類"])
add_df=pd.concat([add_df,dummies],axis=1)
add_df

c:\Users\hakos\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


,薬効分類,year,generic_share_q,generic_share_r,114.0,121.0,123.0,124.0,131.0,132.0,...,321.0,332.0,339.0,391.0,399.0,441.0,449.0,625.0,629.0,799.0
0,114.0,2008,NaN,NaN,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,114.0,2009,NaN,NaN,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,114.0,2010,NaN,NaN,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,114.0,2011,NaN,NaN,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,114.0,2012,NaN,NaN,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,799.0,2011,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
260,799.0,2012,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
261,799.0,2013,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
262,799.0,2022,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [120]:
data[data["generic_share_q"].isnull()]

,薬効分類,year,generic_share_q,generic_share_r,114.0,121.0,123.0,124.0,131.0,132.0,...,321.0,332.0,339.0,391.0,399.0,441.0,449.0,625.0,629.0,799.0
584,229.0,2014,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3224,229.0,2015,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7294,229.0,2016,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11016,224.0,2017,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11285,229.0,2017,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15307,229.0,2018,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19008,224.0,2019,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23016,224.0,2020,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26998,224.0,2021,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [105]:
ids=data.iloc[:,4:]
ids

,114.0,121.0,123.0,124.0,131.0,132.0,211.0,218.0,222.0,223.0,...,321.0,332.0,339.0,391.0,399.0,441.0,449.0,625.0,629.0,799.0
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
85,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
101,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
153,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
213,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29129,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
29169,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
29373,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
29616,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [106]:
ids=data.iloc[:,4:]
X=data[["year"]+list(ids.columns)]
y=data["generic_share_q"]
# X = sm.add_constant(X)  # 定数項を追加
model = sm.OLS(y, X).fit()


In [107]:
data=data.append(add_df,ignore_index=True)
data.sort_values(by=["薬効分類","year"],inplace=True)
data.reset_index(drop=True,inplace=True)
data

C:\Users\hakos\AppData\Local\Temp\ipykernel_11172\1296814082.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(add_df,ignore_index=True)


,薬効分類,year,generic_share_q,generic_share_r,114.0,121.0,123.0,124.0,131.0,132.0,...,321.0,332.0,339.0,391.0,399.0,441.0,449.0,625.0,629.0,799.0
0,114.0,2008,NaN,NaN,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,114.0,2009,NaN,NaN,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,114.0,2010,NaN,NaN,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,114.0,2011,NaN,NaN,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,114.0,2012,NaN,NaN,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,799.0,2019,0.006196,0.024777,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
524,799.0,2020,0.006056,0.022847,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
525,799.0,2021,0.007953,0.028184,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
526,799.0,2022,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [108]:
target="generic_share_q"

In [109]:
target="generic_share_q"
data_est=data.dropna(subset=[target])
ids=data_est.iloc[:,4:]
X=data_est[["year"]+list(ids.columns)]
y=data_est[target]
# X = sm.add_constant(X)  # 定数項を追加
model = sm.OLS(y, X).fit()

# モデルの要約を表示
print(model.summary())
missing=data[data[target].isnull()]
missing=missing[["year"]+list(ids.columns)]
# missing["const"]=1
predicted_values=model.predict(missing)
# predicted_values
missing[target]=predicted_values
data.update(missing)

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [110]:
data_est

,薬効分類,year,generic_share_q,generic_share_r,114.0,121.0,123.0,124.0,131.0,132.0,...,321.0,332.0,339.0,391.0,399.0,441.0,449.0,625.0,629.0,799.0
6,114.0,2014,0.236031,0.053874,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,114.0,2015,0.295651,0.073113,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,114.0,2016,0.323616,0.085162,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,114.0,2017,0.362927,0.097579,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,114.0,2018,0.334082,0.085869,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,799.0,2017,0.005504,0.024843,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
522,799.0,2018,0.005183,0.025933,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
523,799.0,2019,0.006196,0.024777,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
524,799.0,2020,0.006056,0.022847,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
data.to_csv("../merged/len3_ndb_generic_imputed.csv")

C:\Users\hakos\AppData\Local\Temp\ipykernel_11172\1296814082.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(add_df,ignore_index=True)


,薬効分類,year,generic_share_q,generic_share_r,114.0,121.0,123.0,124.0,131.0,132.0,...,321.0,332.0,339.0,391.0,399.0,441.0,449.0,625.0,629.0,799.0
0,114.0,2008,NaN,NaN,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,114.0,2009,NaN,NaN,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,114.0,2010,NaN,NaN,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,114.0,2011,NaN,NaN,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,114.0,2012,NaN,NaN,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,799.0,2019,0.006196,0.024777,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
524,799.0,2020,0.006056,0.022847,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
525,799.0,2021,0.007953,0.028184,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
526,799.0,2022,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
